In [53]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [54]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [55]:
import random
random.seed(42)

from src.utils import TEST_KEY_MAPPING

test_q_keys_mapping = TEST_KEY_MAPPING

# test_q_keys_mapping = {
#     "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
#     "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
#     "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
#     "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
#     "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
# }


# Hyper parameters

In [56]:

survey_names = [
    # "American_Trends_Panel_W26",
    # "American_Trends_Panel_W27",
    # "American_Trends_Panel_W29",
    # "American_Trends_Panel_W32",
    "American_Trends_Panel_W34",
    "American_Trends_Panel_W41",
    "American_Trends_Panel_W82",
]

def skip_exp(filename):

    # check known test
    if 'v17' not in filename or '0904' not in filename: return True


def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    
    # segs = ['history', 'demo', 'persona', 'v11', 'v10', 'prompt3', 'prompt4', 'run1', 'run2', 'run3']
    segs = ['history', 'demo', 'persona', 'withname', 'all' 'allq']
    for level in ['13', '16', '17']: segs.append('v'+level)
    for train in ['200', '500', '2000', 'All']: segs.append('train'+train)
    for top in range(10, 110, 10): segs.append('top'+str(top))
    for skew in ['5', '3', '10']: segs.append('skew'+skew)
    for usedemo in ['True', 'False']: segs.append('usedemo'+usedemo)
    for run in ['run1', 'run2', 'run3']: segs.append(run)
    for setting in ['queryponly', 'querydp', 'querydonly']: segs.append(setting)

    exp_elements = [_ for _ in filename.split('_') if _ in segs]
    final = '_'.join(exp_elements)
    return final

# Main

In [57]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name, sort_by):

    with open(f'experiment/data/human_resp/{survey_name}/user_test_q_key_mapping.json', 'r') as f:
        user_test_q_key_mapping = json.load(f)
    
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]

    for key in res:
        print(key, len(res[key]))

    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    # acc_df.sort_values(by='acc', ascending=False)


    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in survey_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue

        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = user_test_q_key_mapping[str(entry["user_idx"])][q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    if sort_by == 'q_1wd':
        merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    elif sort_by == 'acc':
        merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df

    return merged_df

# Results

In [58]:
from IPython.display import display

for survey_name in survey_names:
    print(survey_name)
    merged = get_full_results(survey_name, sort_by='acc')
    # merged = get_full_results(survey_name, sort_by='q_1wd')
    display(merged)
    print('\n\n\n')

American_Trends_Panel_W34
v17_persona_run1 2
v17_history_persona_run2 2
v17_history_run3 2
v17_history_run1 2
v17_history_demo_persona_run3 2
v17_demo_run2 2
v17_history_demo_run2 2
v17_history_demo_run1 2
v17_demo_run3 2
v17_history_persona_run3 2
v17_history_demo_persona_run2 2
v17_history_run2 2
v17_history_demo_persona_run1 2
v17_persona_run2 2
v17_history_demo_run3 2
v17_history_persona_run1 2
v17_demo_run1 2
Prediction: |humans have evolved over time due to processes such as natural selection; god or a higher power had no role in this process|, Gold: |humans have evolved over time due to processes that were guided or allowed by god or a higher power|, References: |Humans have evolved over time due to processes such as natural selection; God or a higher power had no role in this proc/Humans have evolved over time due to processes that were guided or allowed by God or a higher power/Humans have existed in their present form since the beginning of time/Refused|
Prediction: |humans h

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_history_demo_run1,'=603/1000,0.603,"{1.0: 346, 0.0: 602, 2.0: 13, 3.0: 2, 0.5: 10}",0.393628,14,13
v17_history_demo_run2,'=600/1000,0.6,"{1.0: 347, 0.0: 599, 2.0: 15, 3.0: 2, 0.5: 12}",0.398974,12,13
v17_history_demo_run3,'=599/1000,0.599,"{1.0: 348, 0.0: 598, 2.0: 14, 3.0: 2, 0.5: 11}",0.398253,14,13
v17_history_demo_persona_run3,'=586/1000,0.586,"{1.0: 356, 0.0: 586, 3.0: 3, 2.0: 11, 0.5: 10}",0.405797,21,13
v17_history_demo_persona_run1,'=584/1000,0.584,"{0.0: 584, 1.0: 360, 3.0: 3, 2.0: 10, 0.5: 9}",0.40735,21,13
v17_history_run1,'=582/1000,0.582,"{1.0: 357, 0.0: 581, 2.0: 18, 3.0: 3, 0.5: 10}",0.420021,18,13
v17_history_demo_persona_run2,'=581/1000,0.581,"{0.0: 581, 1.0: 362, 3.0: 3, 2.0: 11, 0.5: 8}",0.411399,22,13
v17_history_run3,'=580/1000,0.58,"{1.0: 359, 0.0: 579, 2.0: 17, 3.0: 3, 0.5: 10}",0.420455,19,13
v17_history_run2,'=578/1000,0.578,"{1.0: 362, 0.0: 577, 2.0: 18, 3.0: 2, 0.5: 10}",0.422085,18,13
v17_history_persona_run3,'=565/1000,0.565,"{1.0: 362, 0.0: 565, 2.0: 15, 3.0: 4, 0.5: 7}",0.427597,34,13






American_Trends_Panel_W41
v17_demo_run1 2
v17_history_demo_run1 2
v17_history_demo_persona_run3 2
v17_history_demo_persona_run2 2
v17_history_run1 2
v17_history_run3 2
v17_history_demo_persona_run1 2
v17_history_run2 2
v17_history_persona_run2 2
v17_history_demo_run3 2
v17_demo_run2 2
v17_history_persona_run1 2
v17_history_persona_run3 2
v17_history_demo_run2 2
v17_demo_run3 2
Prediction: |legal with some restrictions|, Gold: |legal but with some restrictions|, References: |Legal with no restrictions/Legal but with some restrictions/Illegal except in certain cases/Illegal with no exceptions/Refused|
Prediction: |legal with some restrictions|, Gold: |legal but with some restrictions|, References: |Legal with no restrictions/Legal but with some restrictions/Illegal except in certain cases/Illegal with no exceptions/Refused|
Prediction: |legal with some restrictions|, Gold: |legal but with some restrictions|, References: |Legal with no restrictions/Legal but with some restrictions/Ill

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_history_demo_run3,'=32/57,0.561404,"{1.0: 19, 0.0: 33, 0.5: 4, 2.0: 1}",0.403509,0,0
v17_history_demo_run2,'=520/1000,0.52,"{1.0: 333, 0.0: 533, 0.5: 75, 2.0: 37, 1.5: 3,...",0.462869,1,16
v17_history_demo_persona_run2,'=519/1000,0.519,"{1.0: 329, 0.0: 531, 0.5: 74, 2.0: 34, 1.5: 5,...",0.466769,6,16
v17_history_demo_run1,'=518/1000,0.518,"{1.0: 334, 0.0: 532, 0.5: 74, 2.0: 36, 1.5: 3,...",0.462283,3,16
v17_history_demo_persona_run1,'=517/1000,0.517,"{1.0: 328, 0.0: 529, 0.5: 74, 2.0: 36, 1.5: 5,...",0.470317,7,16
v17_history_demo_persona_run3,'=516/1000,0.516,"{1.0: 329, 0.0: 528, 0.5: 75, 2.0: 35, 1.5: 5,...",0.469806,7,16
v17_history_persona_run1,'=509/1000,0.509,"{1.0: 337, 0.0: 521, 0.5: 71, 2.0: 36, 1.5: 5,...",0.473792,11,16
v17_history_persona_run3,'=507/1000,0.507,"{1.0: 337, 0.0: 519, 0.5: 74, 2.0: 37, 1.5: 5,...",0.47641,9,16
v17_history_persona_run2,'=499/1000,0.499,"{1.0: 341, 0.0: 511, 0.5: 75, 2.0: 37, 1.5: 5,...",0.48251,12,16
v17_history_run2,'=497/1000,0.497,"{0.0: 511, 1.0: 335, 0.5: 84, 2.0: 50, 1.5: 2,...",0.493902,0,16






American_Trends_Panel_W82
v17_history_demo_persona_run2 2
v17_history_run1 2
v17_history_persona_run2 2
v17_history_run3 2
v17_history_demo_run3 2
v17_history_demo_persona_run1 2
v17_history_demo_run1 2
v17_history_persona_run3 2
v17_history_run2 2
v17_demo_run2 2
v17_history_demo_run2 2
v17_demo_run3 2
v17_demo_run1 2
v17_history_persona_run1 2
v17_history_demo_persona_run3 2
Prediction: |somewhat serious problem|, Gold: |not very strong conflicts|, References: |Very strong conflicts/Strong conflicts/Not very strong conflicts/There are not conflicts/Refused|
Prediction: |very confident|, Gold: |a lot of confidence|, References: |A lot of confidence/Some confidence/Not too much confidence/No confidence at all/Refused|
Prediction: |refused|, Gold: |somewhat bad job|, References: |Very good job/Somewhat good job/Somewhat bad job/Very bad job/Refused|
Prediction: |very confident|, Gold: |some confidence|, References: |A lot of confidence/Some confidence/Not too much confidence/No conf

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_history_demo_run3,'=391/675,0.579259,"{0.0: 390, 1.0: 237, 2.0: 25, 0.5: 5, 3.0: 3}",0.452273,2,13
v17_history_persona_run2,'=578/1000,0.578,"{0.0: 574, 1.0: 344, 2.0: 31, 0.5: 10, 3.0: 5}",0.441909,17,19
v17_history_persona_run3,'=578/1000,0.578,"{0.0: 574, 1.0: 346, 2.0: 31, 0.5: 11, 3.0: 5}",0.443123,14,19
v17_history_demo_run1,'=575/1000,0.575,"{0.0: 573, 1.0: 355, 2.0: 34, 0.5: 10, 3.0: 3}",0.448205,6,19
v17_history_persona_run1,'=575/1000,0.575,"{0.0: 571, 1.0: 346, 2.0: 33, 0.5: 11, 3.0: 5}",0.447723,15,19
v17_history_demo_run2,'=573/1000,0.573,"{0.0: 571, 1.0: 356, 2.0: 35, 0.5: 10, 3.0: 3}",0.451282,6,19
v17_history_demo_persona_run1,'=570/1000,0.57,"{0.0: 565, 1.0: 361, 2.0: 33, 0.5: 11, 3.0: 4}",0.456366,7,19
v17_history_demo_persona_run3,'=570/1000,0.57,"{0.0: 567, 1.0: 359, 2.0: 32, 0.5: 11, 3.0: 4}",0.452724,8,19
v17_history_run3,'=569/1000,0.569,"{0.0: 568, 1.0: 350, 2.0: 40, 0.5: 11, 3.0: 4}",0.459918,8,19
v17_history_demo_persona_run2,'=568/1000,0.568,"{0.0: 564, 1.0: 361, 2.0: 34, 0.5: 11, 3.0: 4}",0.458419,7,19


In [59]:
print(merged.to_csv())

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v17_history_demo_run3,'=391/675,0.5792592592592593,"Counter({0.0: 390, 1.0: 237, 2.0: 25, 0.5: 5, 3.0: 3})",0.45227272727272727,2,13
v17_history_persona_run2,'=578/1000,0.578,"Counter({0.0: 574, 1.0: 344, 2.0: 31, 0.5: 10, 3.0: 5})",0.44190871369294604,17,19
v17_history_persona_run3,'=578/1000,0.578,"Counter({0.0: 574, 1.0: 346, 2.0: 31, 0.5: 11, 3.0: 5})",0.4431230610134436,14,19
v17_history_demo_run1,'=575/1000,0.575,"Counter({0.0: 573, 1.0: 355, 2.0: 34, 0.5: 10, 3.0: 3})",0.4482051282051282,6,19
v17_history_persona_run1,'=575/1000,0.575,"Counter({0.0: 571, 1.0: 346, 2.0: 33, 0.5: 11, 3.0: 5})",0.4477225672877847,15,19
v17_history_demo_run2,'=573/1000,0.573,"Counter({0.0: 571, 1.0: 356, 2.0: 35, 0.5: 10, 3.0: 3})",0.4512820512820513,6,19
v17_history_demo_persona_run1,'=570/1000,0.57,"Counter({0.0: 565, 1.0: 361, 2.0: 33, 0.5: 11, 3.0: 4})",0.4563655030800821,7,19
v17_history_demo_persona_run3,'=570/1000,0.57,"Counter({0.0: 567, 1.0: 3